In [ ]:
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.linalg import inv
from numpy.random import multivariate_normal
from notebook.services.config import ConfigManager
from traitlets.config.manager import BaseJSONConfigManager
import matplotlib as mpl
import math
from scipy.integrate import odeint


# set jupyter configurations
%matplotlib inline
%config InlineBackend.figure_format='retina'
default_dpi = mpl.rcParamsDefault['figure.dpi']
mpl.rcParams['figure.dpi'] = default_dpi*1.2


# set global variables
gt_color = 'r'
data_color = '#0000FF'
linreg_color = '#519872'
abc_color = 'purple'
sbi_color  = '#DE1A1A'

### 1.4.1 Ballistic motion problem

In [ ]:
def ballistic(launch_velocity, launch_angle, drag_coefficient=1.0000e-6, sample=False):
    """ ... """
    y_t = lambda t: launch_velocity * t * np.sin(math.radians(launch_angle)) - 0.5*9.81*t**2 
    x = np.linspace(0,100,101)
    y = y_t(x)

    samples = y + np.random.randn(x.shape[0]) * 150
   
    plt.plot(x,y, label='Ground truth', color=gt_color)
    if sample:
        plt.scatter(x, samples,label='Samples', marker='x', color=data_color)
    plt.ylim(0)
    plt.xlim(0)
    plt.legend()
    plt.show()
    
    return dict(param=(launch_velocity, launch_angle, drag_coefficient), y=y, samples=samples)


ballistic_motion = interactive(ballistic, launch_velocity=(50,500), launch_angle=(30,60), sample=False)
ballistic_motion 

### 1.4.2 Linear regression as baseline

In [ ]:
# get data from ground truth
ballistic_param = np.array(ballistic_motion.result['param'])
data = ballistic_motion.result['samples']

X = np.linspace(0,100,101)
# # linear regression baseline
features = np.array([X**0, X**1, X**2]).T
linreg_param = inv(features.T @ features).dot(features.T @ data)
linreg_reconstruction = features @ linreg_param

In [ ]:
# plotting
plt.scatter(X, data, marker='x', label='Samples', color=data_color, alpha=0.3)
plt.plot(X, linreg_reconstruction, 'k', label = 'Linear regression', color=linreg_color)

plt.legend()
plt.ylim(0)
plt.xlabel('$x$')
plt.ylabel('$f_x$')
plt.show()

### 1.4.3 Try to reproduce data manually 🛠️

In [ ]:
def ballistic_simulator(param, domain = X):
    y_t = lambda t: param[0] * t * np.sin(math.radians(param[1])) - 0.5*9.81*t**2 
    return y_t(domain) + np.random.randn(domain.shape[0]) #* 250


def distance(distance_function, prediction, data):
    """ Second-order function that takes in a distance function and its arguments and returns the result. """
    return distance_function(prediction, data)



# sample parameters from a uniform distribution
param = np.array([np.random.uniform(200, 350), np.random.uniform(40, 50)])


# pass parameters to simulator and see result
prediction = ballistic_simulator(param, X)
plt.plot(prediction, label='Reconstruction', color=abc_color)
plt.scatter(X, data, marker='x', label='Samples', color=data_color, alpha=0.3)

plt.ylim(0)
plt.legend()




# write own distance functions
mse = lambda prediction, data: np.square(prediction - data).mean()
chebyshev = lambda prediction, data: max([np.abs(data[i]-prediction[i]) for i in range(data.shape[0])])

# what would be a good acceptance threshold?
plt.title(f'distance for $v_0={param[0]:.1f}$ and $\\theta={param[1]:.1f}$: {distance(chebyshev, prediction, data):.2f}')

### 1.4.4 Automate the search for parameters 🛠️

In [ ]:
num_simulations = 100
best_param = None
for _ in range(num_simulations):
    param = np.array([np.random.uniform(50, 500), np.random.uniform(30, 60)])
    # pass to simulator
    prediction = ballistic_simulator(param, X)
    # compute distance
    dist = distance(chebyshev, prediction, data)
    # check if within epsilon
    if dist < 500:
        # if so return parameters
        best_param = param
print(best_param)

In [ ]:
# plot with found parameters
plt.plot(ballistic_simulator(best_param, X), label='Reconstruction', color=abc_color)
plt.scatter(X, data, marker='x', label='Samples', color=data_color, alpha=0.3)

plt.ylim(0)
plt.legend()

- You just discovered rejection ABC!
- Reflect on how choice of $\epsilon$, $d$ was made

### 1.4.5 When does this break?

In [ ]:
def d0_dt(theta, t,b,m,g,l):
    theta1 = theta[0]
    theta2 = theta[1]
    dtheta1_dt = theta2
    dtheta2_dt = -(b/m * theta2) - (g/l * math.sin(theta1))
    dtheta_dt = [ dtheta1_dt, dtheta2_dt]
    
    return dtheta_dt


def pendulum(b,m,l, sample=True):
    theta_0 = [0,3]
    t = np.linspace(0,20,150)    
    # solve ODE
    solution = odeint(d0_dt, theta_0,t,args =(b,m,9.81,l))
    angular_velocity_measurements =  solution[:,0] + np.random.randn(t.shape[0]) * 0.2

    
    # plot angular displacement - also plot angular velocity?
    plt.plot(t, solution[:,0], color = gt_color, linestyle = '-',linewidth = 2,label =r'$ \frac{d\theta_1}{dt}$ = $\theta_2$')
    if sample: # and measurements
        plt.scatter(t,angular_velocity_measurements,marker='x', color=data_color, label='measurements')
    plt.xlabel('$t$')
    plt.grid(True)
    plt.ylabel('Angular displacement $\\theta$')
    plt.legend(loc = 'best')
    plt.show()
    
    return dict(param=(b,m,l), solution=solution, data=angular_velocity_measurements)


damped_pendulum = interactive(pendulum, b=(0,1,0.1), m=(0.01,5,0.1), l=(1,4,0.5), sample=True)
damped_pendulum

### 1.4.6 What can we do about this?
- What summary statistics for this data would come to your  mind?
- Can we learn summary statistics with neural networks?

### 1.4.7 Introduce SBI as a potentially useful approach to solve our problem